In [5]:
import pandas as pd
import numpy as np

f = open("data/data17.txt", "r")
line = f.readlines()[0][:-1]
#line = ">>><<><>><<<>><>>><<<>>><<<><<<>><>><<>>"
print(len(line))

def empty_A():
    A = [["|"] + ['.']*7 + ["|"] for i in range(4)]
    A.insert(0, ["+"] + ['-']*7 + ["+"])
    return A

def print_A(A):
    for l in A[::-1]:
        print("".join(l))
    pass

# shapes are defined as lists of blocks, with positionw relative to leftbottom corner 
shapes = [
    [[0, 0], [1, 0], [2, 0], [3, 0]],
    [[1, 0], [0, 1], [1, 1], [2, 1], [1, 2]],
    [[0, 0], [1, 0], [2, 0], [2, 1], [2, 2]],
    [[0, 0], [0, 1], [0, 2], [0, 3]],
    [[0, 0], [1, 0], [0, 1], [1, 1]]
]

A = empty_A()
print_A(A)

10091
|.......|
|.......|
|.......|
|.......|
+-------+


In [31]:
def get_peak(N_shapes, force_end=False):
    t = 0
    peak = 0
    combs = {}
    A = empty_A()
    d_out = {}

    for k in range(1, N_shapes+1):
        #print("k =", k)
        # create new shape
        corner = (3, peak+4)
        shape = shapes[(k-1)%5]
        positions = [[corner[0]+c[0], corner[1]+c[1]] for c in shape]
        zmax = max([pos[1] for pos in positions])
        #print("zmax =", zmax)
        for i in range(zmax+1-len(A)):
            A.append(["|"] + ['.']*7 + ["|"])

        # drop new shape
        can_drop = True
        while can_drop:
            # lateral shift
            if line[t%len(line)]=='<':
                shift = -1
            elif line[t%len(line)]=='>':
                shift = 1
            else:
                print("Bad readin of wind")
            can_shift = True
            for pos in positions:
                #print(pos, len(A))
                if A[pos[1]][pos[0]+shift]!='.':
                    can_shift = False
            if can_shift:
                for pos in positions:
                    pos[0] += shift

            # vertical shift
            for pos in positions:
                if A[pos[1]-1][pos[0]]!='.':
                    can_drop = False
            if can_drop:
                for pos in positions:
                    pos[1] -= 1

            # moove to next iteration
            t += 1

        # fix the blocks in A
        for pos in positions:
            A[pos[1]][pos[0]] = '#'
            if pos[1] > peak:
                peak = pos[1]

        if (k>1) and (force_end==False):
            # keep the combination
            comb = (t%len(line), (k-1)%5, positions[0][0], last_pos)
            if comb in combs.keys():
                print("loop_start", combs[comb][0], "score_from_loop_start", combs[comb][1])
                d_out["loop_start"] = combs[comb][0]
                d_out["score_from_loop_start"] = combs[comb][1]
                d_out["loop_end"] = k
                d_out["score_from_loop_end"] = peak
                print(comb, "loop_end", k, "score_from_loop_end", peak)
                break
            else:
                combs[comb] = (k, peak)
        
        last_pos = positions[0][0]

    d_out["peak"] = peak
    #d_out["A"] = A
    return d_out

In [33]:
get_peak(10000)

loop_start 47 score_from_loop_start 71
(306, 1, 4, 3) loop_end 1792 score_from_loop_end 2838


{'loop_start': 47,
 'score_from_loop_start': 71,
 'loop_end': 1792,
 'score_from_loop_end': 2838,
 'peak': 2838}

In [34]:
N = 2022 # test 3068, true 3217
print("True answer:", get_peak(N, True)['peak'])
d = get_peak(N)

loop_start = d['loop_start']
score_from_loop_start = d['score_from_loop_start']
loop_end = d['loop_end']
score_from_loop_end = d['score_from_loop_end']
loop_size = loop_end - loop_start

answer = score_from_loop_start + ((N-loop_start) // loop_size) * (score_from_loop_end - score_from_loop_start)
print("after last loop", answer)
residual = (N-loop_start)%loop_size + loop_start
d2 = get_peak(residual)
print("residual", residual, "with peak", d2['peak'])
answer = answer + d2['peak'] - score_from_loop_start

print("my answer:", answer)

True answer: 3217
loop_start 47 score_from_loop_start 71
(306, 1, 4, 3) loop_end 1792 score_from_loop_end 2838
after last loop 2838
residual 277 with peak 450
my answer: 3217


In [35]:
N = 1000000000000
answer = score_from_loop_start + ((N-loop_start) // loop_size) * (score_from_loop_end - score_from_loop_start)
print("after last loop", answer)
residual = (N-loop_start)%loop_size + loop_start
d2 = get_peak(residual)
print("residual", residual, "with peak", d2['peak'])
answer = answer + d2['peak'] - score_from_loop_start

print("my answer:", answer)

after last loop 1585673350905
residual 1010 with peak 1588
my answer: 1585673352422
